In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from helpers.helper import calculate_terms, calculate_write_off_rate, calculate_interest_metrics, calculate_avg_days_in_arrears

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
loans = pd.read_csv('~\Desktop\iohk_docs\_loans.csv')
payments = pd.read_csv('~\Desktop\iohk_docs\_payments.csv')
borrowers = pd.read_csv('~\Desktop\iohk_docs\_borrowers.csv')

C:\Users\vladb\AppData\Local\Temp\ipykernel_26104\3427265589.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  loans = pd.read_csv('~\Desktop\iohk_docs\_loans.csv')


## Loans

In [5]:
loans_omit_cols = ['as_of_datetime', 'issue_date', 'maturity_date', 'closing_date', 'write_off_date', 'default_date']
loans = loans.apply(lambda x: x.astype('category') if x.dtype == 'object' and x.name not in loans_omit_cols else x)

In [6]:
invalid_entry = loans[loans['maturity_date'] == '0019-03-20']
invalid_entry.head(1)

,as_of_datetime,loan_id,borrower_id,issue_date,maturity_date,closing_date,currency,principal_amount,interest_rate,write_off_date,write_off_amount,default_date,default_amount,product_name,penalties,fees,total_outstanding
1481,2024-02-01 00:00:00,S0VfS1VfMjA2MDA5,S0VfS1VfMTYxMTA1OTUyN2d3WXZyOVZoVVg=,2021-01-19,0019-03-20,2021-03-31,KES,40000.0,0.0,2021-03-31,40000.0,NaN,NaN,ProLoan,0.0,0.0,40000.0


In [7]:
invalid_maturity_dates = loans[
    (loans['maturity_date'].isna()) | 
    (loans['maturity_date'].astype(str) < loans['issue_date'].astype(str))
]
loans = loans[~loans['loan_id'].isin(invalid_maturity_dates['loan_id'])]


In [8]:
loans_date_omit_col = ['as_of_datetime']

loans = loans.apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d') if x.dtype == 'object' and x.name not in loans_date_omit_col else x)
loans['as_of_datetime'] = pd.to_datetime(loans['as_of_datetime'])

In [9]:
loans.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 2757862 entries, 0 to 2757981
Data columns (total 17 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   as_of_datetime     2757862 non-null  datetime64[ns]
 1   loan_id            2757862 non-null  category      
 2   borrower_id        2745040 non-null  category      
 3   issue_date         2757862 non-null  datetime64[ns]
 4   maturity_date      2757862 non-null  datetime64[ns]
 5   closing_date       2757680 non-null  datetime64[ns]
 6   currency           2757862 non-null  category      
 7   principal_amount   2757862 non-null  float64       
 8   interest_rate      2757862 non-null  float64       
 9   write_off_date     2315778 non-null  datetime64[ns]
 10  write_off_amount   2315778 non-null  float64       
 11  default_date       210 non-null      datetime64[ns]
 12  default_amount     210 non-null      float64       
 13  product_name       2757862 non-n

## Payments

In [10]:
payment_omit_col = ['payment_date']
payments = payments.apply(lambda x: x.astype('category') if x.dtype == 'object' and x.name not in payment_omit_col else x)

In [11]:
payments['payment_date'] = pd.to_datetime(payments['payment_date'], format='%Y-%m-%d')

In [12]:
payments = payments.dropna(subset=['payment_date'])

In [13]:
payments.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 5681730 entries, 0 to 5681805
Data columns (total 6 columns):
 #   Column        Non-Null Count    Dtype         
---  ------        --------------    -----         
 0   payment_id    5681653 non-null  category      
 1   payment_date  5681730 non-null  datetime64[ns]
 2   loan_id       5681730 non-null  category      
 3   currency      5681730 non-null  category      
 4   amount        5681730 non-null  float64       
 5   type          5681730 non-null  category      
dtypes: category(4), datetime64[ns](1), float64(1)
memory usage: 398.4 MB


## Borrowers

In [14]:
borrowers = borrowers.sort_values(by='as_of_datetime', ascending=False)
borrowers = borrowers.drop_duplicates(subset=['borrower_id'], keep='first')

In [15]:
borrowers = borrowers[(borrowers['date_of_birth'] >= '1924-01-01') & (borrowers['date_of_birth'] <= '2005-07-01')]

borrowers[['as_of_datetime', 'date_of_birth']] = borrowers[['as_of_datetime', 'date_of_birth']].apply(pd.to_datetime)

In [16]:
borrower_omit_col = ['as_of_datetime', 'date_of_birth']
borrowers = borrowers.apply(lambda x: x.astype('category') if x.dtype == 'object' and x.name not in borrower_omit_col else x)

In [17]:
borrowers.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 428861 entries, 4282096 to 242941
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   as_of_datetime      428861 non-null  datetime64[ns]
 1   borrower_id         428861 non-null  category      
 2   gender              428861 non-null  category      
 3   country             428861 non-null  category      
 4   geo_classification  140929 non-null  category      
 5   date_of_birth       428861 non-null  datetime64[ns]
dtypes: category(4), datetime64[ns](2)
memory usage: 32.1 MB


## Master data

In [18]:
master_data = pd.merge(
    pd.merge(loans, payments, how='left', on='loan_id'),
    borrowers, how='left', on='borrower_id'
)

In [19]:
master_data = master_data.loc[:, ~master_data.columns.str.endswith('_y')]
master_data = master_data.rename(columns=lambda x: x.replace('_x', '') if x.endswith('_x') else x)

In [20]:
master_data = master_data.sort_values(by=['loan_id', 'as_of_datetime'], ascending=[True, False]) 

In [21]:
invalid_borrower_loans = master_data[master_data['borrower_id'].isna() & master_data['loan_id'].notna()]
invalid_borrower_loans.shape

(13414, 25)

In [22]:
master_data.head()

,as_of_datetime,loan_id,borrower_id,issue_date,maturity_date,closing_date,currency,principal_amount,interest_rate,write_off_date,write_off_amount,default_date,default_amount,product_name,penalties,fees,total_outstanding,payment_id,payment_date,amount,type,gender,country,geo_classification,date_of_birth
1434765,2024-10-01,S0VfS1VfIDI2NDUwMg==,S0VfS1VfMTYyNDYxMzEyOGN2S2tKTFZqeFk=,2022-02-12,2022-02-26,2022-04-30,KES,8000.0,0.075,2022-04-30,8960.0,NaT,NaN,ProLoan,360.0,0.0,8960.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT
1434760,2024-09-01,S0VfS1VfIDI2NDUwMg==,S0VfS1VfMTYyNDYxMzEyOGN2S2tKTFZqeFk=,2022-02-12,2022-02-26,2022-04-30,KES,8000.0,0.075,2022-04-30,8960.0,NaT,NaN,ProLoan,360.0,0.0,8960.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT
1434759,2024-08-01,S0VfS1VfIDI2NDUwMg==,S0VfS1VfMTYyNDYxMzEyOGN2S2tKTFZqeFk=,2022-02-12,2022-02-26,2022-04-30,KES,8000.0,0.075,2022-04-30,8960.0,NaT,NaN,ProLoan,360.0,0.0,8960.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT
1434758,2024-07-01,S0VfS1VfIDI2NDUwMg==,S0VfS1VfMTYyNDYxMzEyOGN2S2tKTFZqeFk=,2022-02-12,2022-02-26,2022-04-30,KES,8000.0,0.075,2022-04-30,8960.0,NaT,NaN,ProLoan,360.0,0.0,8960.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT
1434767,2024-06-01,S0VfS1VfIDI2NDUwMg==,S0VfS1VfMTYyNDYxMzEyOGN2S2tKTFZqeFk=,2022-02-12,2022-02-26,2022-04-30,KES,8000.0,0.075,2022-04-30,8960.0,NaT,NaN,ProLoan,360.0,0.0,8960.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT


In [23]:
master_data.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 2883610 entries, 1434765 to 1616093
Data columns (total 25 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   as_of_datetime      2883610 non-null  datetime64[ns]
 1   loan_id             2883610 non-null  object        
 2   borrower_id         2870196 non-null  object        
 3   issue_date          2883610 non-null  datetime64[ns]
 4   maturity_date       2883610 non-null  datetime64[ns]
 5   closing_date        2883428 non-null  datetime64[ns]
 6   currency            2883610 non-null  category      
 7   principal_amount    2883610 non-null  float64       
 8   interest_rate       2883610 non-null  float64       
 9   write_off_date      2440303 non-null  datetime64[ns]
 10  write_off_amount    2440303 non-null  float64       
 11  default_date        210 non-null      datetime64[ns]
 12  default_amount      210 non-null      float64       
 13  product_nam

## Metrics

In [24]:
metrics_df = master_data.copy() 

In [25]:
metrics_df = calculate_terms(metrics_df)

### Write-off rate

In [26]:
#dropping duplicates and keeping most recent instance of loan_id to calculate write_off amount & gross yield
metrics_df = metrics_df.drop_duplicates(subset=['loan_id'], keep='first') 

In [27]:
#dropping out of bounds maturity_date
metrics_df = metrics_df[metrics_df['loan_id'] != 'VUdfS1VfNDI2NQ==']

write_off_rate = calculate_write_off_rate(metrics_df)
print(f'Write off rate is: {write_off_rate:.2f}%')

Write off rate is: 32.37%


### Gross yield

In [28]:
metrics_df = calculate_interest_metrics(metrics_df)

In [29]:
total_actual_interest = metrics_df['actual_interest'].sum()
total_principal_amount = metrics_df['principal_amount'].sum()

gross_yield = (total_actual_interest / total_principal_amount) * 100
print(f'Gross yield is: {gross_yield:.2f}%')

Gross yield is: 46.85%


### Collections Rate

In [30]:
# df_no_write_offs = metrics_df[metrics_df['write_off_date'].isna()]
# valid_payments = df_no_write_offs[(df_no_write_offs['amount'].notnull()) & (df_no_write_offs['total_outstanding'] < 0)]

# total_collections = valid_payments['amount'].sum()
# total_outstanding = metrics_df['total_outstanding'].sum()

# collections_rate = total_collections / total_outstanding * 100

### AVG days in arrears

In [31]:
avg_days_in_arrears = calculate_avg_days_in_arrears(metrics_df)

print(f"Average Days in Arrears: {avg_days_in_arrears:.2f} days")

Average Days in Arrears: 21.74 days
